In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from qutip import *
from scipy.optimize import curve_fit

plt.rc('font', size=20)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=20)
plt.rc('xtick', labelsize=15)
plt.rc('ytick', labelsize=15)
plt.rc('legend', fontsize=15)
plt.rc('figure', titlesize=20)

In [2]:
def readFile(filename):
    timeTickFirst, timeTickLast, x, ion3, ion3_raw, ion3_bottom, ion3_top, ion4, ion4_raw, ion4_bottom, \
    ion4_top, ion5, ion5_raw, ion5_bottom, ion5_top, ion2, ion2_raw, ion2_bottom, ion2_top, ion6, ion6_raw, \
    ion6_bottom, ion6_top\
      = np.loadtxt(filename, comments='#', delimiter='\t', converters=None,
                   skiprows=0,unpack=True, encoding='latin1')
    return x, ion4

In [3]:
df_dep = pd.read_excel('data_VAET_heat.xlsx')
df_dep

filename_nbar2p0 filename_nbar0p0
0       nbar2p0_001      nbar0p0_001
1       nbar2p0_002              NaN
2       nbar2p0_003              NaN
3       nbar2p0_004              NaN
4       nbar2p0_005              NaN
5       nbar2p0_006              NaN
6       nbar2p0_007              NaN
7       nbar2p0_008              NaN
8       nbar2p0_009              NaN
9       nbar2p0_010              NaN
10      nbar2p0_011              NaN
11      nbar2p0_012              NaN
12      nbar2p0_013              NaN
13      nbar2p0_014              NaN
14      nbar2p0_015              NaN
15      nbar2p0_016              NaN
16      nbar2p0_017              NaN
17      nbar2p0_018              NaN
18      nbar2p0_019              NaN
19      nbar2p0_020              NaN

In [4]:
def get_pops(params, option='c'):
    Nmax = params['Nmax']
    H = params['eps']/2 * tensor(sigmaz(), qeye(Nmax))
    H += params['Delta']/2 * tensor(sigmax(), qeye(Nmax))
    H += params['kappa']/2 * tensor(sigmaz(), (destroy(Nmax) + create(Nmax)))
    H += params['nu'] * tensor(qeye(2), create(Nmax) * destroy(Nmax))
    init_state = tensor(ket2dm(basis(2,0)), thermal_dm(Nmax, params['nbar']))
    tlist = params['tlist']
    e_op = tensor(ket2dm(basis(2,0)), qeye(Nmax))
    e_op2 = tensor(qeye(2), create(Nmax) * destroy(Nmax))
    if option == 'c':
        c_ops = [np.sqrt(params['Gamma'] * (params['nbar'] + 1)) * tensor(qeye(2), destroy(Nmax))]
        if params['nbar'] > 0:
            c_ops += [np.sqrt(params['Gamma'] * params['nbar']) * tensor(qeye(2), create(Nmax))]
    elif option == 'd':
        c_ops = [np.sqrt(params['Gamma']) * tensor(qeye(2), create(Nmax) * destroy(Nmax))]
    result = mesolve(H, init_state, tlist, e_ops=[e_op, e_op2], c_ops=c_ops)
    pops = result.expect[0]
    phonons = result.expect[1]
    return pops, phonons

In [5]:
sim_time = 12 * 2*np.pi
steps = 200
tlist = np.linspace(0, sim_time, steps+1)

params = {'nu': 1.04, 'eps': 1., 'Delta': 0.3, 'kappa': 0.3, 'Gamma': 0.000, 'nbar': 0.,
              'tlist': tlist, 'Nmax': 10}
pops_sim_coh, _ = get_pops(params, option='d')

params = {'nu': 1.04, 'eps': 1., 'Delta': 0.3, 'kappa': 0.3, 'Gamma': 0.007, 'nbar': 0.,
              'tlist': tlist, 'Nmax': 10}
pops_sim_coh_noise, _ = get_pops(params, option='d')

params = {'nu': 1.04, 'eps': 1., 'Delta': 0.3, 'kappa': 0.3, 'Gamma': 0.000, 'nbar': 2.,
              'tlist': tlist, 'Nmax': 10}
pops_sim_heat, _ = get_pops(params, option='d')

params = {'nu': 1.04, 'eps': 1., 'Delta': 0.3, 'kappa': 0.3, 'Gamma': 0.007, 'nbar': 2.,
              'tlist': tlist, 'Nmax': 10}
pops_sim_heat_noise, _ = get_pops(params, option='d')

In [6]:
def analyze(data_list):
    expdata_all = np.array(data_list).reshape((len(data_list), -1))
    expdata = np.mean(expdata_all, axis=0)
    
    variance = np.std(expdata_all, axis=0)**2 / len(data_list)
    shotnoise = np.maximum(np.sqrt(expdata_all * (1-expdata_all) / 100), 1e-3)
    variance = variance + np.mean(shotnoise**2, axis=0)
    errorbar = np.sqrt(variance)
    
    return expdata, errorbar

coherent_pop_list_dep = []
theory_list = []
for seed in np.arange(100, 120):
    with open('./VAETnoisy_nbar=2.0_no=1.npy', 'rb') as f:  # seed starts with 100
        pops = np.load(f)
    theory_list += [pops[seed-100,:]]
theory_pop, error_theory  = analyze(theory_list)

heated_list = []
for fname in df_dep['filename_nbar2p0'][:20]:
    filename = fname
    _, pop = readFile(filename)
    heated_list += [pop[:50]]
heated_pop, error_heated  = analyze(heated_list)

for fname in df_dep['filename_nbar0p0']:
    if not pd.isna(fname):
        filename = fname
        _, pop = readFile(filename)
        coherent_pop_list_dep += [pop[:50]]
Coh_pop, error_coh  = analyze(coherent_pop_list_dep)


In [7]:
## Unit Conversion ##

Delta_cm = 100 #cm^-1
Delta_eV = Delta_cm/8065.73 # eV
hbar = 6.62607015e-34/2/np.pi
q = 1.60217663e-19
Delta = Delta_eV*q/hbar # Hz * 2pi
print('Delta: 2 pi '+str(Delta/1e12/2/np.pi)+' THz')
scale = 1e12/(Delta/2/np.pi) #1e12 ps / (freq in Hz)
print(scale)

tlist = tlist*scale

Delta: 2 pi 2.9978554155025736 THz
0.3335717909638933


In [8]:
color_list = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple']
plt.figure(figsize = (8,2.4))
t_exp = tlist[:-1]/2/np.pi
plt.errorbar(t_exp[::4], Coh_pop, yerr=error_coh,\
                fmt='o', alpha=0.5,markersize=5, color=color_list[2], ecolor=color_list[2],\
                mfc=color_list[2])
plt.errorbar(t_exp[::4], heated_pop, yerr=error_heated,\
                fmt='o', alpha=0.5,markersize=5, color=color_list[3], ecolor=color_list[3],\
                mfc=color_list[3])
plt.plot(tlist/2/np.pi, pops_sim_coh, color = color_list[2],label=r'$\bar{n} = 0.0$')
plt.plot(tlist/2/np.pi, pops_sim_coh_noise, color = color_list[2], ls = '--')

plt.plot(tlist/2/np.pi, pops_sim_heat, color = color_list[3], label=r'$\bar{n} = 2.0$')
plt.plot(tlist/2/np.pi, pops_sim_heat_noise, color = color_list[3], ls = '--')
plt.legend()
plt.grid(True)
plt.xlabel(r'Time (ps)')
plt.ylabel('Donor pop.')

plt.savefig('Figure5_2.png', dpi=300, bbox_inches='tight')